<a href="https://colab.research.google.com/github/zshamroukh/Colab/blob/main/api_use_case_phx_income_ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contextual Insights in 3 Steps via the Parcl Labs API

#Step 1: Register for the API

You can retrieve Parcl Labs API data with only a couple lines of GraphQL code. For demonstration purposes here we will show you a use case using Python, one of the most popular programming languages for data analysis, but the API can be used in combination with your favorite tech stack. Once you’ve received your key, by signing up [here](https://www.parcllabs.com/contact?utm_source=ParclLabs&utm_medium=Blog&utm_campaign=API-USE-CASE-BLOG) for free, you can store it as a variable in Python along with the API URL:

In [ ]:
import os
import requests
import pandas as pd
import plotly.express as px

# Storing the API credentials as an environment variable
url = "https://api-pilot.parcllabs.com/v1/graphql"
bearer_token = os.environ['PARCL_LABS_API_PILOT_KEY']

# Step 2: Define the GraphQL query

Leveraging the Parcl Labs API, we can provide context local context on the Phoenix real estate market by examining the income to price feed ratio in each city in the MSA. Context specific insights allows home buyers and investors to make informed decisions about the market.

First we name our query, in this case PHX_INCOME_BY_PLPF, and then we can query all cities within the “Phoenix-Mesa-Chandler, AZ” MSA from the CITY table (see the Parcl Labs API [docs](https://docs.parcllabs.com/docs/msa) for a comprehensive list of geographies and objects available in the API).

After defining the MSA and level of geography, the columns we output from the CITY table are CITY_NAME and PARCL_ID (our unique identifier of different levels of geography). Within the CITY geographies is a nested table, census, containing the income data. In the example below we query census at the geography we defined with a parameter of year equal to 2020 (there are census values for every year and we only want one population value for each category). The the price feed table is also nested under the CITY geography so we lastly grab an average of the price feed over the last 365 days for each city. This is what the query would look like:

In [ ]:
#Storing the query to get income and the avergae Parcl Price Feed by city over the last year
query = """
query PHX_INCOME_BY_PLPF {
  CITY(where: {MSA_NAME: {_eq: "Phoenix-Mesa-Chandler, AZ"}}) {
    CITY_NAME
    PARCL_ID
    census(where: {YEAR: {_eq: 2020}}) {
      INCOME_MEDIAN
    }
    parcl_price_feed_aggregate(order_by: {DATE: desc}, limit: 365) {
      aggregate {
      	avg {
          PARCL_PRICE_FEED
        }
      }
    }
	}
}
"""

#Step 3: Call the API and visualize the results

Once you have plugged in your query, the URL and API key you can call the API with Python:

In [ ]:
# Call the API using our query and credentials
response = requests.post(
   url=url,
   json={
     "query": query
   },
   headers={
     "Authorization": f"Bearer {bearer_token}",
     'content-type': "application/json"
   }
)
 
# Parse the JSON response to native Python object
out = response.json()
data = out['data']['CITY']

The next step is to flatten the JSON response into a dataframe and divide income by the price feed to get the income to price feed ratio:

In [ ]:
#Pull the data into a dataframe and find the ratio
PHX_PPF_INCOME_RATIO_df = pd.json_normalize(data, 'census', ['CITY_NAME', 'PARCL_ID', ['parcl_price_feed_aggregate', 'aggregate', 'avg', 'PARCL_PRICE_FEED']])
PHX_PPF_INCOME_RATIO_df['Income Price Feed Ratio'] = PHX_PPF_INCOME_RATIO_df['INCOME_MEDIAN'] / PHX_PPF_INCOME_RATIO_df['parcl_price_feed_aggregate.aggregate.avg.PARCL_PRICE_FEED']
PHX_PPF_INCOME_RATIO_df = PHX_PPF_INCOME_RATIO_df.sort_values(by=['Income Price Feed Ratio'], ascending=False).head(15)

print(PHX_PPF_INCOME_RATIO_df)

    INCOME_MEDIAN      CITY_NAME PARCL_ID  \
42          45390         Hayden  5386848   
24          40081      Winkelman  5386851   
82          44775    Dudleyville  5386993   
37          44775        Mammoth  5386881   
15          52292         Kearny  5386880   
60          62885    Wintersburg  5387252   
70          52721         Aguila  5386890   
48          65741       Red Rock  5387092   
23          96737    Citrus Park  5386930   
9           70543     Morristown  5387061   
46          49469   Campo Bonito  5386920   
47          40769     San Manuel  5387103   
50          62099  Cactus Forest  5386967   
72          57578       Wittmann  5387253   
64          52292       Superior  5386885   

   parcl_price_feed_aggregate.aggregate.avg.PARCL_PRICE_FEED  \
42                                          18.355911          
24                                          33.939221          
82                                          48.705614          
37                     

And finally visualize it with your choice of visualization libraries, here we use [Plotly](https://plotly.com/):

In [ ]:
#Plot the ratio for the 15 most affordable Phoenix Cities
PHX_Ratio_fig = px.bar(PHX_PPF_INCOME_RATIO_df, x='CITY_NAME',
                y='Income Price Feed Ratio',
                title='Income to Parcl Labs Price Feed Ratio: Phoenix MSA',
                width=900, height=500,
                labels={"CITY_NAME": "City"}
                )

PHX_Ratio_fig.show()